## Postwork 6

In [220]:
import requests
import pandas as pd
import numpy as np

In [221]:
# Establecemos la liga de la api y los paramtros a usar
endpoint = "https://www.climatewatchdata.org/api/v1/data/historical_emissions"
parameters = {'regions': ['MEX'],
              'start_year': 1990,
              'end_year': 2018,
              'sort_col': 'year',
              'sort_dir': 'DESC'}

### Automatizamos la llamada de la api para verificar el estado de nuestro reuqests

In [222]:
#Verificamos que salio bien nuestro requests
try:
    r = requests.get(endpoint, parameters)
    if r.status_code == 200:
        json = r.json()
        data = json['data']
        print('Petición exitosa')
except:
    print('Ocurrió un error')

Petición exitosa


In [223]:
json = r.json()
json.keys()

dict_keys(['data', 'meta'])

In [233]:
# Normalizamos el jdataset de una estructura tipo JSON a una de tipo Data Frame
data = json['data']
normalized = pd.json_normalize(data)
df = pd.DataFrame.from_dict(normalized)
df.head()

,id,iso_code3,country,data_source,sector,gas,unit,emissions
0,658228,MEX,Mexico,CAIT,Total excluding LUCF,All GHG,MtCO₂e,"[{'year': 2011, 'value': 664.66}, {'year': 199..."
1,658229,MEX,Mexico,CAIT,Total including LUCF,All GHG,MtCO₂e,"[{'year': 2003, 'value': 492.69}, {'year': 201..."
2,658230,MEX,Mexico,CAIT,Energy,All GHG,MtCO₂e,"[{'year': 2017, 'value': 495.63}, {'year': 199..."
3,658231,MEX,Mexico,CAIT,Industrial Processes,All GHG,MtCO₂e,"[{'year': 2003, 'value': 21.35}, {'year': 1990..."
4,658232,MEX,Mexico,CAIT,Agriculture,All GHG,MtCO₂e,"[{'year': 1997, 'value': 85.51}, {'year': 1998..."


### Exploración y limpieza del dataset

Observemos que los sectores no son necesarios para nuestro análisis, por lo que, nos quedaremos unicamente con la fila que incluye a todos.

In [234]:
including = df[df["sector"] == "Total including LUCF"]
including = incluyente.reset_index(drop = True)
including

,id,iso_code3,country,data_source,sector,gas,unit,emissions
0,658229,MEX,Mexico,CAIT,Total including LUCF,All GHG,MtCO₂e,"[{'year': 2003, 'value': 492.69}, {'year': 201..."
1,658243,MEX,Mexico,CAIT,Total including LUCF,CO2,MtCO₂e,"[{'year': 2010, 'value': 402.21}, {'year': 199..."
2,658255,MEX,Mexico,CAIT,Total including LUCF,CH4,MtCO₂e,"[{'year': 2018, 'value': 145.06}, {'year': 201..."
3,658264,MEX,Mexico,CAIT,Total including LUCF,N2O,MtCO₂e,"[{'year': 2007, 'value': 37.65}, {'year': 1990..."
4,658273,MEX,Mexico,CAIT,Total including LUCF,F-Gas,MtCO₂e,"[{'year': 2012, 'value': 16.29}, {'year': 2011..."


In [235]:
including = df[df["sector"] == "Total including LUCF"]
including = incluyente.reset_index(drop=True)

Observemos la columna de emisiones no pudo ser normalizada, sin emabargo está claro que es una lista de diccionarios, por lo que vamos a extraer estos datos por separado.
Para ello es necesario acceder a los valores de cada diccionario, lo cual se realiza en el siguiente código:

In [236]:
# Observemos los elementos del diccionario: dos llaves('year' y 'value') y sus respectivos valores
print(including['emissions'][0][0].items())
print(including['emissions'][0][0].keys())
including['emissions'][0][0].values()

dict_items([('year', 2003), ('value', 492.69)])
dict_keys(['year', 'value'])


dict_values([2003, 492.69])

In [237]:
#Con get() podemos acceder al valor de cada llave del diccionario
print(including['emissions'][0][0].get('value'))
including['emissions'][0][0].get('year')

492.69


2003

In [238]:
# Definimos una funcion para extraer los datos

def extraccion(lista, pais):

  # Definimos listas donde almacenaremos los datos extraidos (años, valor, pais)
  lista_1 =[]
  lista_2 = []
  paises = []

  for i in range(len(lista)):
    for y in range(len(lista[i])):

      # Extraemos el pais
      paises.append(pais[i])
 
      # Extraemos los años
      anio = lista[i][y].get('year')
      lista_1.append(anio)

      # Extraemos los valores
      valor = lista[i][y].get('value')
      lista_2.append(valor)

  # Unimos en un df
  data = pd.DataFrame({'country': paises, 'year': lista_1, 'value': lista_2})

  return data

Antes de usar la función observemos el tamaño de cada lista de diccionarios, la cual es de 29 datos, además se observa que cada lista corresponde a un gas y como son 5 gases (incluyendo la suma de gases "ALL GHG"), al final nuestro nuevo dataset debe ser de un tamaño de 145 registros, pues es el resultado de 29 diccionarios por 5 variables de gases

In [239]:
#Tamaño de cada registro de la columna emissiones
len(including["emissions"][0])

29

In [240]:
# Usamos la función y extraemos nuestro nuevo dataset
data = extraccion(including["emissions"], including["country"])
data

,country,year,value
0,Mexico,2003,492.69
1,Mexico,2016,698.62
2,Mexico,2015,684.99
3,Mexico,2014,659.98
4,Mexico,2013,668.85
...,...,...,...
140,Mexico,1994,1.74
141,Mexico,1993,1.99
142,Mexico,1992,1.70
143,Mexico,1991,1.97


Observemos que el tamaño del nuevo dataset es de 145 filas, lo cual coincide con la explicación atnerior de que cada lista es de un tamaño de 29 diccionarios multiplicado por 5 gases

Ahora agregamos los gases, que como mencionamos son 29 registros los cuales corresponden a cada gas por lo que es suficiente si los añadimos en orden 29 veces cada uno.

In [243]:
# Creamos una lista con todos los 145 datos de los gases
lista = ["all","co2", "ch4", "n2o", "fgas"]

lista_gases = []
for gases in lista:
  for i in range(29):
    lista_gases.append(gases)


In [244]:
# Agregamos una columna a nuestro dataset llamada emissions y le asignamos los valores de la lista anterior
data["emissions"] = lista_gases
data.head()

,country,year,value,emissions
0,Mexico,2003,492.69,all
1,Mexico,2016,698.62,all
2,Mexico,2015,684.99,all
3,Mexico,2014,659.98,all
4,Mexico,2013,668.85,all


In [245]:
#Filtramos por años y vemos que se tienen los 5 gaases para cada año del país correspondiente
data[data['year'] == 2011]

,country,year,value,emissions
6,Mexico,2011,680.75,all
53,Mexico,2011,494.81,co2
79,Mexico,2011,130.57,ch4
108,Mexico,2011,39.11,n2o
117,Mexico,2011,16.26,fgas


In [184]:
data["value"][data["year"] == 2011].loc[0]

664.66

In [247]:
# Observemos que no hay datos nulos
data.isna().sum()

country      0
year         0
value        0
emissions    0
dtype: int64

**Nota**: Observese que este dataset es complementario al dataset que se está utilizando en la investigación, ya que la api utilizada corresponde a una de loas fuentes de información utilizada para el poryecto final, y se puede observar que en ambos datasets al final se tienen las 5 variables de gases por año y por país. 

Sin embargo, no se utiliza específicamente este dataset debido a que el que se está usando se le agregaron otros dos conjuntos de datos de diferentes fuentes de información, estos fueron los niveles de temperatra y los fallecimientos por enferemedades respiratorias, lo cual ya se había realizado para antes de este postwork.